## Feature Extraction

In [ ]:
from scipy.spatial import distance as dist
import numpy as np
import pandas as pd
import math

In [ ]:
def eye_aspect_ratio(eye):
	# compute the euclidean distances between the two sets of
	# vertical eye landmarks (x, y)-coordinates
	A = dist.euclidean(eye[1], eye[5])
	B = dist.euclidean(eye[2], eye[4])
	# compute the euclidean distance between the horizontal
	# eye landmark (x, y)-coordinates
	C = dist.euclidean(eye[0], eye[3])
	# compute the eye aspect ratio
	ear = (A + B) / (2.0 * C)
	# return the eye aspect ratio
	return ear

In [ ]:
def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[14], mouth[18])
    C = dist.euclidean(mouth[12], mouth[16])
    mar = (A ) / (C)
    return mar

In [ ]:
def circularity(eye):
    A = dist.euclidean(eye[1], eye[4])
    radius  = A/2.0
    Area = math.pi * (radius ** 2)
    p = 0
    p += dist.euclidean(eye[0], eye[1])
    p += dist.euclidean(eye[1], eye[2])
    p += dist.euclidean(eye[2], eye[3])
    p += dist.euclidean(eye[3], eye[4])
    p += dist.euclidean(eye[4], eye[5])
    p += dist.euclidean(eye[5], eye[0])
    return 4 * math.pi * Area /(p**2)

In [ ]:
def average_ear(left_eye, right_eye):
  return np.mean([left_eye, right_eye])

In [ ]:
def mouth_over_eye(ear, mar):
  mouth_eye = mar/ear
  return mouth_eye

In [ ]:
landmarks_13_15 = np.load('features_13_to_15.npy')
landmarks_16_18 = np.load('features_16_to_18.npy')
individuals_13_15 = np.load('individuals_13_to_15.npy')
individuals_16_18 = np.load('individuals_16_to_18.npy')
labels_13_15 = np.load('labels_13_to_15.npy')
labels_16_18 = np.load('labels_16_to_18.npy')

In [ ]:
print(landmarks_13_15.shape)
print(landmarks_16_18.shape)
print(individuals_13_15.shape)
print(individuals_16_18.shape)
print(labels_13_15.shape)
print(labels_16_18.shape)

(1887, 68, 2)
(1767, 68, 2)
(1887,)
(1767,)
(1887,)
(1767,)


In [ ]:
# Combine both files
landmarks_all = np.concatenate((landmarks_13_15,landmarks_16_18))
individuals_all = np.concatenate((individuals_13_15,individuals_16_18))
labels_all = np.concatenate((labels_13_15,labels_16_18))

In [ ]:
extracted_features = []

for mark in landmarks_all:
  left_eye = mark[36:42]
  right_eye = mark[42:48]
  mouth = mark[48:68]
  left_EAR = eye_aspect_ratio(left_eye)
  right_EAR = eye_aspect_ratio(right_eye)
  left_PUC = circularity(left_eye)
  right_PUC = circularity(right_eye)
  PUC = np.mean([left_PUC,right_PUC])
  MAR = mouth_aspect_ratio(mouth)
  EAR = average_ear(left_EAR, right_EAR)
  MOE = mouth_over_eye(EAR,MAR)
  features = [EAR, MAR, MOE, PUC]
  extracted_features.append(features)



  

In [ ]:
image_features = pd.DataFrame(np.asarray(extracted_features), columns=['EAR', 'MAR', 'MOE', 'PUC'])
image_features['labels'] = labels_all
image_features['individuals'] = individuals_all

In [ ]:
image_features.head()

,EAR,MAR,MOE,PUC,labels,individuals
0,0.275309,0.016793,0.060997,0.461757,0,13
1,0.303325,0.134450,0.443253,0.498823,0,13
2,0.306427,0.090308,0.294714,0.505097,0,13
3,0.310777,0.125623,0.404223,0.489660,0,13
4,0.318539,0.047389,0.148770,0.515814,0,13


In [ ]:
image_features.groupby(['individuals','labels']).mean()

EAR       MAR       MOE       PUC
individuals labels                                        
13          0       0.306881  0.041948  0.135803  0.500680
            5       0.324143  0.023881  0.075861  0.517647
            10      0.255030  0.016363  0.066660  0.426759
14          0       0.346411  0.030084  0.089253  0.550386
            5       0.260380  0.032602  0.125777  0.442671
            10      0.345198  0.026691  0.080933  0.543494
15          0       0.260990  0.013733  0.055568  0.439973
            5       0.265163  0.019825  0.075549  0.434708
            10      0.196825  0.017213  0.089630  0.369482
16          0       0.349603  0.029186  0.082693  0.512677
            5       0.358714  0.092891  0.256682  0.528429
            10      0.292913  0.084225  0.295863  0.450662
17          0       0.334104  0.010339  0.030722  0.523092
            5       0.298407  0.009407  0.034728  0.474774
            10      0.280566  0.018462  0.070467  0.448681
18          0       0.255909  0.044286  0.177535  0.443114
            5       0.241340  0.116419  0.508609  0.404238
            10      0.197509  0.083002  0.465390  0.367675

In [ ]:
# Number of frames in each video per individual
image_features.groupby(['individuals','labels'])['EAR'].count()

individuals  labels
13           0         202
             5         230
             10        207
14           0         205
             5         221
             10        208
15           0         204
             5         204
             10        206
16           0         158
             5         188
             10        201
17           0         202
             5         203
             10        202
18           0         201
             5         198
             10        214
Name: EAR, dtype: int64

The video with lowest number of frames has 158 frames. Standardize frames from all videos to be the same number

In [ ]:
ind_13_0 = image_features[(image_features['individuals'] == 13) & (image_features['labels'] ==  0)].iloc[:158, :]
ind_13_5 = image_features[(image_features['individuals'] == 13) & (image_features['labels'] ==  5)].iloc[:158, :]
ind_13_10 = image_features[(image_features['individuals'] == 13) & (image_features['labels'] ==  10)].iloc[:158, :]
ind_14_0 = image_features[(image_features['individuals'] == 14) & (image_features['labels'] ==  0)].iloc[:158, :]
ind_14_5 = image_features[(image_features['individuals'] == 14) & (image_features['labels'] ==  5)].iloc[:158, :]
ind_14_10 = image_features[(image_features['individuals'] == 14) & (image_features['labels'] ==  10)].iloc[:158, :]
ind_15_0 = image_features[(image_features['individuals'] == 15) & (image_features['labels'] ==  0)].iloc[:158, :]
ind_15_5 = image_features[(image_features['individuals'] == 15) & (image_features['labels'] ==  5)].iloc[:158, :]
ind_15_10 = image_features[(image_features['individuals'] == 15) & (image_features['labels'] ==  10)].iloc[:158, :]
ind_16_0 = image_features[(image_features['individuals'] == 16) & (image_features['labels'] ==  0)].iloc[:158, :]
ind_16_5 = image_features[(image_features['individuals'] == 16) & (image_features['labels'] ==  5)].iloc[:158, :]
ind_16_10 = image_features[(image_features['individuals'] == 16) & (image_features['labels'] ==  10)].iloc[:158, :]
ind_17_0 = image_features[(image_features['individuals'] == 17) & (image_features['labels'] ==  0)].iloc[:158, :]
ind_17_5 = image_features[(image_features['individuals'] == 17) & (image_features['labels'] ==  5)].iloc[:158, :]
ind_17_10 = image_features[(image_features['individuals'] == 17) & (image_features['labels'] ==  10)].iloc[:158, :]
ind_18_0 = image_features[(image_features['individuals'] == 18) & (image_features['labels'] ==  0)].iloc[:158, :]
ind_18_5 = image_features[(image_features['individuals'] == 18) & (image_features['labels'] ==  5)].iloc[:158, :]
ind_18_10 = image_features[(image_features['individuals'] == 18) & (image_features['labels'] ==  10)].iloc[:158, :]


In [ ]:
# Create separate frame for each individual
ind_13 = pd.concat([ind_13_0,ind_13_10, ind_13_5])
ind_14 = pd.concat([ind_14_0, ind_14_10, ind_14_5])
ind_15 = pd.concat([ind_15_0,ind_15_10, ind_15_5])
ind_16 = pd.concat([ind_16_0, ind_16_10, ind_16_5])
ind_17 = pd.concat([ind_17_0, ind_17_10, ind_17_5])
ind_18 = pd.concat([ind_18_0, ind_18_10, ind_18_5])

In [ ]:
def normalization(feature, feature_mean, std_dev):
  norm_feature =  (feature - feature_mean) / std_dev
  return norm_feature

In [ ]:
# Extract standard deviation and mean of the first 4 frames of alert video for each individual
dataframes = [ind_13, ind_14, ind_15, ind_16, ind_17, ind_18]
for frame in dataframes:
  column_means = frame.head(4).iloc[:, :4].mean()
  column_std = frame.head(4).iloc[:, :4].std()
  frame['NORM_EAR'] = frame['EAR'].apply(lambda x: normalization(x, column_means[0], column_std[0]))
  frame['NORM_MAR'] = frame['MAR'].apply(lambda x: normalization(x, column_means[1], column_std[1]))
  frame['NORM_MOE'] = frame['MOE'].apply(lambda x: normalization(x, column_means[2], column_std[2]))
  frame['NORM_PUC'] = frame['PUC'].apply(lambda x: normalization(x, column_means[3], column_std[3]))

In [ ]:
ind_13.head(10)

,EAR,MAR,MOE,PUC,labels,individuals,NORM_EAR,NORM_MAR,NORM_MOE,NORM_PUC
0,0.275309,0.016793,0.060997,0.461757,0,13,-1.472587,-1.401509,-1.395918,-1.415283
1,0.303325,0.134450,0.443253,0.498823,0,13,0.271834,0.797101,0.829263,0.522114
2,0.306427,0.090308,0.294714,0.505097,0,13,0.464930,-0.027757,-0.035409,0.850004
3,0.310777,0.125623,0.404223,0.489660,0,13,0.735822,0.632165,0.602064,0.043164
4,0.318539,0.047389,0.148770,0.515814,0,13,1.219064,-0.829776,-0.884979,1.410191
5,0.287774,0.115144,0.400119,0.486421,0,13,-0.696458,0.436334,0.578172,-0.126146
6,0.361872,0.000000,0.000000,0.554691,0,13,3.917145,-1.715315,-1.750995,3.442203
7,0.291807,0.021930,0.075154,0.461131,0,13,-0.445336,-1.305510,-1.313511,-1.448009
8,0.345893,0.079295,0.229247,0.542693,0,13,2.922238,-0.233558,-0.416505,2.815099
9,0.325720,0.033537,0.102962,0.515904,0,13,1.666168,-1.088625,-1.151633,1.414907


In [ ]:
# Combine all frames
all_frames = pd.concat([ind_13, ind_14,ind_15, ind_16, ind_17,ind_18])


In [ ]:
# Preview data set
all_frames.head()

,EAR,MAR,MOE,PUC,labels,individuals,NORM_EAR,NORM_MAR,NORM_MOE,NORM_PUC
0,0.275309,0.016793,0.060997,0.461757,0,13,-1.472587,-1.401509,-1.395918,-1.415283
1,0.303325,0.134450,0.443253,0.498823,0,13,0.271834,0.797101,0.829263,0.522114
2,0.306427,0.090308,0.294714,0.505097,0,13,0.464930,-0.027757,-0.035409,0.850004
3,0.310777,0.125623,0.404223,0.489660,0,13,0.735822,0.632165,0.602064,0.043164
4,0.318539,0.047389,0.148770,0.515814,0,13,1.219064,-0.829776,-0.884979,1.410191


In [ ]:
# Drop rows where class == 5
all_frames.drop(index=all_frames.loc[all_frames['labels'] == 5].index, inplace=True)
# Convert target variable into a binary variable where 0 is alert and 1 is not alert
all_frames['labels'] = np.where(all_frames['labels'] == 0, 0, 1)

In [ ]:
# Save data to csvs
all_frames.to_csv('frames_13_18.csv', index=False)
